In [ ]:
# 联合预测 
#
# 配置描述     |基础运算            |联合统计       |隐匿查询          |安全求交        |特征工程           |联合建模   |联合预测 
# 数据方       |两方, 三方          |两方          |两方              |两方            |两方              |两方       |两方 
# 算法类型     |加法, 乘法, 比较    |方差, 中位数   |高效率, 高隐匿性   |平衡, 非平衡    |WOE和IV计算        |LR        |LR
# 测试数据集   |数据集α             |数据集α       |数据集β            |数据集γ        |epsilon           |epsilon    |epsilon

import os
path = './ds'
assert os.path.exists(path)

In [ ]:
%%time

import pyarrow as pa 
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pandas as pd
import numpy as np

ds1 = ds.dataset(f'{path}/ds_epsilon1_test')
epsilon_y = ds1.to_table(columns={'0': ds.field('0')})['0'].to_pandas()
epsilon_y = epsilon_y.replace(-1, 0)
proj = {col:ds.field(col) for col in ds1.schema.names}
proj.pop('0')
epsilon1 = ds1.to_table(columns=proj)
epsilon2 = ds.dataset(f'{path}/ds_epsilon2_test').to_table()
epsilon_X = pd.DataFrame({col: data.to_pandas() for col, data 
        in zip(epsilon1.column_names+epsilon2.column_names, epsilon1.columns+epsilon2.columns)})

In [ ]:
%%time

import xgboost as xgb
# Create DMatrix
dtest = xgb.DMatrix(epsilon_X, epsilon_y)
model = xgb.Booster({'nthread': 4})
model.load_model(f"{path}/epsilon_model.json")

In [ ]:
%%time

# Predict results
prediction = model.predict(dtest)

# Print prediction results
print(f'Prediction results:\n{prediction}')